In [ ]:
# Importing modules
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential, Input, Model
from tensorflow.keras.layers import MaxPooling2D, Conv2D, Flatten, Dense, Dropout, Average
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

os.chdir('/kaggle/input/petfinder-pawpularity-score')

In [ ]:
# Loading meta data and response variable
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

## Approach

In a previous attempt I used a basic ensemble of a Convolutional Neural Network (CNN) and a Deep Nerual Network (DNN), trained on the image data and meta data respectively. This attempt didn't yield the desired result with a Root Mean Squared Error (RMSE) of $\approx$ 20.44, so I will now try a different approach. One of the problems with the previous attempt was, that DNN had a hard time minimizing loss i.e., predicting the `Pawpularity` based on the meta data. This led me to thinking "maybe there isn't great coherence between the meta data and response variable", which is why I will now try an approach solely based on the image data. 

For this attempt I will create an ensemble model consisting of three CNN's with varying filters in the convolutional layers and neurons in the hidden layer before output.  

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Showing first 5 images
for i in range(1, 6):
    img = mpimg.imread(os.path.join('train/', os.listdir('train/')[i]))
    plt.subplot(1,5,i)
    plt.imshow(img)
    plt.axis('off')
plt.show()

In [ ]:
import PIL
from PIL import Image

widths = []
heights = []

for img in os.listdir("train/"):
    img_path = os.path.join("train/", img) # Making image file path
    im = Image.open(img_path)
    widths.append(im.size[0])
    heights.append(im.size[1])

AVG_HEIGHT = round(sum(heights)/len(heights))
AVG_WIDTH = round(sum(widths)/len(widths))

In [ ]:
df['file_name'] = df['Id'].astype(str) + '.jpg' 
test['file_name'] = test['Id'].astype(str) + '.jpg'

In [ ]:
image_size = [int(AVG_HEIGHT/3), int(AVG_WIDTH/3)]
input_shape = image_size + [3]
batch = 32

train_datagen = ImageDataGenerator(samplewise_center=True, 
                                   samplewise_std_normalization=True,
                                   validation_split = 0.1)

test_datagen = ImageDataGenerator(samplewise_center=True,
                                  samplewise_std_normalization=True,)

train_generator = train_datagen.flow_from_dataframe(
    dataframe = df,
    directory = 'train/',
    x_col = 'file_name',
    y_col = 'Pawpularity',
    batch_size = batch,
    shuffle = True,
    subset = 'training',
    color_mode = 'rgb',
    class_mode = 'raw',
    target_size = image_size)

valid_generator = train_datagen.flow_from_dataframe(
    dataframe = df,
    directory = 'train/',
    x_col = 'file_name',
    y_col = 'Pawpularity',
    batch_size = batch,
    shuffle = True,
    subset = 'validation',
    color_mode = 'rgb',
    class_mode = 'raw',
    target_size = image_size)

test_generator = test_datagen.flow_from_dataframe(
    dataframe = test,
    directory = 'test/',
    x_col = 'file_name',
    batch_size = batch,
    shuffle = False,
    color_mode = 'rgb',
    class_mode = None,
    target_size = image_size)

In [ ]:
def cnn_model(num_filters, neurons, input_shape):
    model = Sequential([
        Conv2D(num_filters[0], (3,3), activation='relu', input_shape=input_shape, name='conv_layer_1'),
        MaxPooling2D(2,2),
        
        Conv2D(num_filters[1], (3,3), activation='relu', name='conv_layer_2'),
        MaxPooling2D(2,2),
        
        Conv2D(num_filters[2], (3,3), activation='relu', name='conv_layer_3'),
        MaxPooling2D(2,2),
        
        Conv2D(num_filters[3], (3,3), activation='relu', name='conv_layer_4'),
        MaxPooling2D(2,2),
        
        Flatten(),
        Dense(neurons, activation='relu', name='hidden_layer'),
        Dropout(.5),
        
        Dense(1, name = 'output')
    ])
    
    model.compile(
        loss = 'mse',
        optimizer=Adam(),
        metrics=[RootMeanSquaredError()])
    return model

In [ ]:
params = np.array([[16, 32, 64, 128, 128],
                   [32, 64, 64, 128, 128],
                   [32, 64, 128, 256, 512]])

for i in range(3):
    model = cnn_model(params[i, 0:4].astype(int), params[i, 4], input_shape)
    
    STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
    STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
    
    checkpoint_filepath = '../../working/tmp/checkpoint-model_' + str(i+1)
    
    model_checkpoint_callback = ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_root_mean_squared_error',
        mode='min',
        save_best_only=True) # Saving best checkpoint
    
    earlystopping_callback = EarlyStopping(
        monitor='val_loss',
        patience=5) # stopping training when val_loss doesn't decrease in 5 epochs
    
    reducelronplateau_callback = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3) # reducing learning rate when val_loss doesn't improve for 3 epochs
    
    model.fit(
            train_generator,
            steps_per_epoch=STEP_SIZE_TRAIN,
            validation_data=valid_generator,
            validation_steps=STEP_SIZE_VALID,
            epochs=100,
            callbacks=[model_checkpoint_callback, earlystopping_callback, reducelronplateau_callback])

In [ ]:
models = []
for i in range(3):
    filepath = '../../working/tmp/checkpoint-model_' + str(i+1)
    model = cnn_model(params[i, 0:4].astype(int), params[i, 4], input_shape)
    model.load_weights(filepath)
    models.append(model)

In [ ]:
# Ensemble model
model_input = Input(shape=(input_shape))
model_outputs = [model(model_input) for model in models]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output)



In [ ]:
tf.keras.utils.plot_model(ensemble_model, to_file='../../working/model.png', show_shapes=True)

In [ ]:
# Making submission file
STEP_SIZE_TEST = test_generator.n // 8
pred = ensemble_model.predict(test_generator, steps=STEP_SIZE_TEST)
results = pd.DataFrame({'Id':test['Id']})
results['Pawpularity'] = pred

os.chdir('../../working')

results.to_csv('submission.csv', index=False)